In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:
# ===== Cell 1: Setup =====
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# GPU確認
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# データパス確認
DATA_DIR = '/kaggle/input/dogs-vs-cats-redux-kernels-edition'
print(os.listdir(DATA_DIR))

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda
['sample_submission.csv', 'train.zip', 'test.zip']


In [3]:
# ===== Cell 2: Config =====
class CFG:
    seed = 42
    model_name = 'efficientnet_b3'  # 変更
    img_size = 300                   # 変更
    batch_size = 16                  # メモリ対策で減らす
    epochs = 10                      # 変更
    lr = 1e-4
    num_workers = 2

# 再現性
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CFG.seed)

In [4]:
# ===== Cell 3: Dataset =====
class DogCatDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        if self.labels is not None:
            label = self.labels[idx]
            return img, torch.tensor(label, dtype=torch.float32)
        else:
            return img

In [5]:
# ===== Cell 4: Data Preparation =====
import zipfile

# train.zipを解凍
train_zip = os.path.join(DATA_DIR, 'train.zip')
with zipfile.ZipFile(train_zip, 'r') as z:
    z.extractall('/kaggle/working/')

# test.zipを解凍
test_zip = os.path.join(DATA_DIR, 'test.zip')
with zipfile.ZipFile(test_zip, 'r') as z:
    z.extractall('/kaggle/working/')

# ファイルリスト作成
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

train_files = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

# ラベル抽出（dog=1, cat=0）
train_labels = [1 if 'dog' in os.path.basename(f) else 0 for f in train_files]

print(f"Train: {len(train_files)} images")
print(f"Test: {len(test_files)} images")
print(f"Dogs: {sum(train_labels)}, Cats: {len(train_labels) - sum(train_labels)}")

Train: 25000 images
Test: 12500 images
Dogs: 12500, Cats: 12500


In [6]:
# ===== Cell 5: Train/Val Split =====
from sklearn.model_selection import train_test_split

train_paths, val_paths, train_y, val_y = train_test_split(
    train_files, train_labels,
    test_size=0.2,
    stratify=train_labels,
    random_state=CFG.seed
)

print(f"Train: {len(train_paths)}, Val: {len(val_paths)}")

Train: 20000, Val: 5000


In [7]:
# ===== Cell 6: Transform & DataLoader =====
# timmのモデル用transform
model_temp = timm.create_model(CFG.model_name, pretrained=False)
config = resolve_data_config({}, model=model_temp)
train_transform = create_transform(**config, is_training=True)
val_transform = create_transform(**config, is_training=False)

# Dataset
train_dataset = DogCatDataset(train_paths, train_y, train_transform)
val_dataset = DogCatDataset(val_paths, val_y, val_transform)
test_dataset = DogCatDataset(test_files, labels=None, transform=val_transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

In [8]:
# ===== Cell 7: Model =====
model = timm.create_model(
    CFG.model_name,
    pretrained=True,
    num_classes=1
)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)

# スケジューラー追加（学習率を徐々に下げる）
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=CFG.epochs
)

print(f"Model: {CFG.model_name}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Model: efficientnet_b3
Parameters: 10,697,769


In [9]:
# ===== Cell 8: Training =====
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({'loss': loss.item(), 'acc': correct/total})
    
    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validation'):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return running_loss / total, correct / total

# 訓練実行
best_val_acc = 0
for epoch in range(CFG.epochs):
    print(f"\n===== Epoch {epoch+1}/{CFG.epochs} =====")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)
    scheduler.step()  # スケジューラー更新
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    print(f"LR: {scheduler.get_last_lr()[0]:.6f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Best model saved! Val Acc: {val_acc:.4f}")

# ベストモデルを読み込んで推論
model.load_state_dict(torch.load('best_model.pth'))
print(f"\nBest Val Acc: {best_val_acc:.4f}")


===== Epoch 1/10 =====


Validation: 100%|██████████| 313/313 [00:23<00:00, 13.34it/s]


Train Loss: 0.2124, Train Acc: 0.9339
Val Loss: 0.0292, Val Acc: 0.9914
LR: 0.000098
Best model saved! Val Acc: 0.9914

===== Epoch 2/10 =====


Validation: 100%|██████████| 313/313 [00:23<00:00, 13.34it/s]


Train Loss: 0.0837, Train Acc: 0.9673
Val Loss: 0.0237, Val Acc: 0.9926
LR: 0.000090
Best model saved! Val Acc: 0.9926

===== Epoch 3/10 =====


...

Train Loss: 0.0365, Train Acc: 0.9849
Val Loss: 0.0189, Val Acc: 0.9956
LR: 0.000010

===== Epoch 9/10 =====


Validation: 100%|██████████| 313/313 [00:23<00:00, 13.35it/s]


Train Loss: 0.0329, Train Acc: 0.9868
Val Loss: 0.0179, Val Acc: 0.9962
LR: 0.000002
Best model saved! Val Acc: 0.9962

===== Epoch 10/10 =====


Validation: 100%|██████████| 313/313 [00:23<00:00, 13.31it/s]


Train Loss: 0.0328, Train Acc: 0.9863
Val Loss: 0.0187, Val Acc: 0.9960
LR: 0.000000

Best Val Acc: 0.9962


In [10]:
# ===== Cell 9: Inference & Submission =====
model.eval()
predictions = []

with torch.no_grad():
    for images in tqdm(test_loader, desc='Inference'):
        images = images.to(device)
        outputs = model(images).squeeze(1)
        probs = torch.sigmoid(outputs).cpu().numpy()
        predictions.extend(probs)

# 提出ファイル作成
test_ids = [int(os.path.basename(f).split('.')[0]) for f in test_files]
submission = pd.DataFrame({
    'id': test_ids,
    'label': predictions
})
submission = submission.sort_values('id')
submission.to_csv('submission.csv', index=False)

print(submission.head(10))
print(f"\nSubmission saved: {len(submission)} rows")

Inference: 100%|██████████| 782/782 [00:58<00:00, 13.48it/s]

       id         label
2006    1  9.999993e-01
714     2  1.000000e+00
2822    3  1.000000e+00
4905    4  1.000000e+00
4833    5  6.296742e-08
7093    6  4.130951e-05
5178    7  2.230560e-17
2562    8  1.537125e-07
11072   9  6.457912e-09
4698   10  1.464308e-10

Submission saved: 12500 rows
